In [1]:
import os

In [2]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_checkpoint: Path

    num_train_epochs: int
    learning_rate: float
    warmup_steps: int
    per_device_train_batch_size: int
    gradient_accumulation_steps: int
    weight_decay: float


    optim: str
    lr_scheduler_type: str


    use_cpu: bool
    use_fp16: bool
    tf32: bool
    gradient_checkpointing: bool
    dataloader_pin_memory: bool
    dataloader_num_workers: int
    torch_empty_cache_steps: int


    logging_strategy: str
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_strategy: str
    save_steps: int


In [ ]:
from src.summarizer.utils.common import create_directories
from src.summarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def get_model_trainer_config(self) -> ModelTrainerConfig:
            config = read_yaml(self.config.model_trainer)
            params = read_yaml(self.params.model_trainer)

            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(root_dir=config.root_dir,
                                                    data_path=config.data_path,
                                                    model_checkpoint=config.model_checkpoint,
                                                    num_train_epochs=params.num_train_epochs,
                                                    learning_rate=params.learning_rate,
                                                    warmup_steps=params.warmup_steps,
                                                    per_device_train_batch_size=params.per_device_train_batch_size,
                                                    per_device_eval_batch_size=params.per_device_eval_batch_size,
                                                    gradient_accumulation_steps=params.gradient_accumulation_steps,
                                                    weight_decay=params.weight_decay,
                                                    optim=params.optim,
                                                    lr_scheduler_type=params.lr_scheduler_type,
                                                    use_cpu=params.use_cpu,
                                                    use_bf16=params.bf16,
                                                    seed=params.seed,
                                                    data_seed=params.data_seed,
                                                    tf32=params.tf32,
                                                    gradient_checkpointing=params.gradient_checkpointing,
                                                    dataloader_pin_memory=params.dataloader_pin_memory,
                                                    dataloader_num_workers=params.dataloader_num_workers,
                                                    torch_empty_cache_steps=params.torch_empty_cache_steps,
                                                    logging_strategy=params.logging_strategy,
                                                    logging_steps=params.logging_steps,
                                                    eval_strategy=params.eval_strategy,
                                                    eval_steps=params.eval_steps,
                                                    save_strategy=params.save_strategy,
                                                    save_steps=params.save_steps,
                                                    report_to=params.report_to,
                                                    load_best_model_at_end=params.load_best_model_at_end,
                                                    generation_num_beams=params.generation_num_beams,
                                                    generation_max_length=params.generation_max_length,
                                                    predict_with_generate=params.predict_with_generate,
                                                    train_fraction=params.train_fraction,
                                                    eval_fraction=params.eval_fraction,
                                                    max_grad_norm=params.max_grad_norm,
                                                    torch_compile=params.torch_compile)
            
            return model_trainer_config

In [ ]:
from src.summarizer.entity.config_entity import ModelTrainerConfig
import torch
import evaluate 
import numpy as np 

torch.backends.cuda.matmul.allow_tf32 = True 
torch.backends.cudnn.allow_tf32 = True 

class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config


    def compute_metrics(self, eval_preds):
        preds, labels = eval_preds

        # Handle tuple output
        if isinstance(preds, tuple):
            preds = preds[0]

        # Convert to numpy safely
        preds = np.array(preds)
        labels = np.array(labels)

        # Replace invalid values
        preds[preds < 0] = self.tokenizer.pad_token_id
        preds[preds >= self.tokenizer.vocab_size] = self.tokenizer.pad_token_id

        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)

        # Force safe dtype
        preds = preds.astype(np.int64)
        labels = labels.astype(np.int64)

        decoded_preds = self.tokenizer.batch_decode(
            preds.tolist(),   #critical: convert to Python lists
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        decoded_labels = self.tokenizer.batch_decode(
            labels.tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        decoded_preds = ["\n".join(p.split(". ")) for p in decoded_preds]
        decoded_labels = ["\n".join(l.split(". ")) for l in decoded_labels]

        rouge = evaluate.load("rouge")  # Load rouge here
        
        result = rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True,
        )

        result = {k: round(v * 100, 4) for k, v in result.items()}

        gen_lens = [np.count_nonzero(p != self.tokenizer.pad_token_id) for p in preds]
        result["gen_len"] = float(np.mean(gen_lens))

        return result

    def train(self):

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_checkpoint)
        self.tokenizer = tokenizer
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_checkpoint)

        data_collator = DataCollatorForSeq2Seq(tokenizer=self.tokenizer,
                                               model=model)

        dataset_dict = load_from_disk(self.config.data_path)

        train_frac = self.config.train_fraction
        eval_frac = self.config.eval_fraction

        train_dataset = (
            dataset_dict["train"]
            .shuffle(seed=42)
            .select(range(int(len(dataset_dict["train"]) * train_frac)))
        )

        eval_dataset = (
            dataset_dict["validation"]
            .shuffle(seed=42)
            .select(range(int(len(dataset_dict["validation"]) * eval_frac)))
        )

        training_args = Seq2SeqTrainingArguments(
            output_dir=self.config.root_dir,

            # Core training
            num_train_epochs=self.config.num_train_epochs,
            learning_rate=self.config.learning_rate,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            weight_decay=self.config.weight_decay,

            # Optimizer & scheduler
            optim=self.config.optim,
            lr_scheduler_type=self.config.lr_scheduler_type,

            # Precision & performance
            bf16=self.config.use_bf16,
            tf32=self.config.tf32,
            dataloader_pin_memory=self.config.dataloader_pin_memory,
            dataloader_num_workers=self.config.dataloader_num_workers,

            # Logging / eval / saving
            logging_strategy=self.config.logging_strategy,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.eval_strategy,
            eval_steps=self.config.eval_steps,
            save_strategy=self.config.save_strategy,
            save_steps=self.config.save_steps,

            # Misc
            report_to=self.config.report_to,  # avoids wandb unless you want it
            load_best_model_at_end=self.config.load_best_model_at_end,
            generation_num_beams=self.config.generation_num_beams,
            generation_max_length=self.config.generation_max_length,
            predict_with_generate=self.config.predict_with_generate,
            max_grad_norm=self.config.max_grad_norm,
            torch_compile=self.config.torch_compile
        )

        trainer = Seq2SeqTrainer(model=model,
                          args=training_args,
                          processing_class=self.tokenizer,
                          data_collator=data_collator,
                          train_dataset=train_dataset,
                          eval_dataset=eval_dataset,
                          compute_metrics=self.compute_metrics)
        
        trainer.train()

        model.save_pretrained(os.path.join(self.config.root_dir, "model"))
        self.tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))